## Importing the Library Packages

**DataSet:https://www.kaggle.com/datasets/christianlillelund/csgo-round-winner-classification**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [2]:
data = pd.read_csv("../input/csgo-round-winner-classification/csgo_round_snapshots.csv")

data.head()

In [21]:
data.info()

In [22]:
data.describe(
)

In [3]:
data.isna().any().sum()

In [4]:
data.drop(data.select_dtypes(np.number), axis=1)

In [5]:
data["bomb_planted"] = data["bomb_planted"].astype(np.int16)

In [6]:
encoder = LabelEncoder()

data["map"] = encoder.fit_transform(data["map"])
map_mappings = {index: label for index, label in enumerate(encoder.classes_)}

In [7]:
map_mappings

In [8]:
encoder = LabelEncoder()

data["round_winner"] = encoder.fit_transform(data["round_winner"])
winner_mappings = {index: label for index, label in enumerate(encoder.classes_)}

In [9]:
winner_mappings

In [10]:
y = data["round_winner"]
X = data.drop("round_winner", axis=1)

In [11]:
scaler = RobustScaler()
X = scaler.fit_transform(X)

pd.DataFrame(X)

In [12]:
pca = PCA(n_components=91)
pca.fit(X)

In [13]:
plt.figure(figsize=(10, 10))
plt.hist(pca.explained_variance_ratio_, bins=84)
plt.show()

In [14]:
def getKComponents(pca, alpha):
    total_variance = 0
    for feature, variance in enumerate(pca.explained_variance_ratio_):
        total_variance += variance
        if (total_variance >= 1-alpha):
            return feature+1
    return len(pca.explained_variance_ratio_)


In [15]:
getKComponents(pca, 0.05)

In [16]:
K = getKComponents(pca, 0.05)

In [17]:
X = pca.transform(X)[:, 0:K]
pd.DataFrame(X)

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)

In [19]:
log_model = LogisticRegression(verbose=True)
nn_model = MLPClassifier(verbose=True)

log_model.fit(X_train, y_train)
nn_model.fit(X_train, y_train)

In [20]:
print(f"--Logistic Model: {log_model.score(X_val, y_val)}")
print(f"Neural Net Model: {nn_model.score(X_val, y_val)}")